# Loading the Model

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# Recreate the model architecture
class DeepfakeDetector(nn.Module):
    def __init__(self):
        super(DeepfakeDetector, self).__init__()
        self.model = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.IMAGENET1K_V1)
        self.model.classifier = nn.Sequential(
            nn.Linear(self.model.classifier[1].in_features, 2048),
            nn.ReLU(),
            nn.Dropout(0.8),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Dropout(0.8),
            nn.Linear(2048, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepfakeDetector().to(device)

# Load saved weights
model.load_state_dict(torch.load("PATH TO MODEL/Sample_new_1.8L_deepfake_detector_linkedin.pth", map_location=device))
model.eval()  # Set model to evaluation mode

print("Model loaded successfully!")


## Convert image  to .jpg

In [ ]:
from PIL import Image
import cv2
import os

def convert_to_jpg(input_path, output_path=None):
    # Load the image
    img = Image.open(input_path)

    # Convert if image has an alpha channel (transparency)
    if img.mode in ("RGBA", "LA") or (img.mode == "P" and "transparency" in img.info):
        background = Image.new("RGB", img.size, (255, 255, 255))  # White background
        img = Image.alpha_composite(background, img.convert("RGBA"))

    # Define output path
    if output_path is None:
        output_path = os.path.splitext(input_path)[0] + ".jpg"

    # Save as JPG
    img.convert("RGB").save(output_path, "JPEG", quality=95)
    print(f"Converted: {input_path} → {output_path}")

# Example usage
convert_to_jpg("PATH TO INPUT IMAGE.jpg")  # Change to your file path


## Preprocess the Image

### Upscale

In [ ]:
import os
import cv2
import shutil
from realesrgan import RealESRGANer
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url

# Define Real-ESRGAN upscaler model
def load_upscaler(model_name='RealESRGAN_x4plus'):
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
    model_path = os.path.join('weights', f'{model_name}.pth')

    if not os.path.isfile(model_path):
        model_url = f'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/{model_name}.pth'
        model_path = load_file_from_url(url=model_url, model_dir='weights', progress=True, file_name=None)

    upscaler = RealESRGANer(scale=4, model_path=model_path, model=model, tile=0, tile_pad=10, pre_pad=0, half=False)
    return upscaler

# Upscale images smaller than 512x512
def upscale_image(input_path, output_path, upscaler):
    img = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)
    h, w = img.shape[:2]

    if h >= 512 and w >= 512:
        print(f"Skipping {input_path} (Already >= 512x512)")
        shutil.copy(input_path, output_path)
        return

    scale_h = 512 / h
    scale_w = 512 / w
    scale = min(scale_h, scale_w)

    print(f"Upscaled {input_path} with scale factor {scale:.2f}")
    output, _ = upscaler.enhance(img, outscale=scale)
    output = cv2.resize(output, (512, 512), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(output_path, output)

# Function to process a single image
def process_image(input_path, output_path):
    upscaler = load_upscaler()

    img = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)
    h, w = img.shape[:2]

    if h < 512 or w < 512:
        upscale_image(input_path, output_path, upscaler)
    else:
        print(f"Image {input_path} is already 512x512 or larger, copying without changes.")
        shutil.copy(input_path, output_path)

# Example usage
input_image_path = "PATH TO INPUT IMAGE.jpg"  # Replace with your input image path
output_image_path = "PATH TO INPUT IMAGE_scaledup.jpg"  # Replace with your output image path

process_image(input_image_path, output_image_path)


### Downscale

In [ ]:
from PIL import Image
import shutil

# Function to downscale images larger than 512x512
def downscale_image(input_path, output_path, target_size=(512, 512)):
    img = Image.open(input_path).convert("RGB")
    w, h = img.size

    if w <= 512 and h <= 512:
        print(f"Skipping {input_path} (Already <= 512x512)")
        shutil.copy(input_path, output_path)
        return

    print(f"Downscaling {input_path}: Original size {w}x{h} → Resizing to {target_size[0]}x{target_size[1]}")
    img.thumbnail(target_size, Image.LANCZOS)

    new_img = Image.new("RGB", target_size, (0, 0, 0))
    new_w, new_h = img.size
    paste_x = (target_size[0] - new_w) // 2
    paste_y = (target_size[1] - new_h) // 2
    new_img.paste(img, (paste_x, paste_y))

    new_img.save(output_path)

# Example usage
input_image_path = "PATH TO INPUT IMAGE_scaledup.jpg"  # Replace with your input image path
output_image_path = "PATH TO INPUT IMAGE_scaleddown.jpg"  # Replace with your output image path

downscale_image(input_image_path, output_image_path)


## Testing the Model

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# Preprocessing
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Predict a Single Image
def predict_image(model, image_path):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image).item()
    print(f"Prediction Score: {output:.4f} ({'Fake' if output > 0.5 else 'Real'})")

# Example usage:
predict_image(model, "PATH TO INPUT IMAGE_scaleddown.jpg")


# Performance Metrics

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os

# Define Model Class
class DeepfakeDetector(nn.Module):
    def __init__(self):
        super(DeepfakeDetector, self).__init__()
        self.model = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.IMAGENET1K_V1)
        self.model.classifier = nn.Sequential(
            nn.Linear(self.model.classifier[1].in_features, 2048),
            nn.ReLU(),
            nn.Dropout(0.8),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Dropout(0.8),
            nn.Linear(2048, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

# Load the Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepfakeDetector().to(device)
model.load_state_dict(torch.load("PATH TO MODEL/Sample_new_1.8L_deepfake_detector_linkedin.pth", map_location=device))
model.eval()

# Define Dataset Class
class DeepfakeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        for label, subdir in enumerate(["Real", "Fake"]):
            subdir_path = os.path.join(root_dir, subdir)
            if os.path.exists(subdir_path):
                for img_name in os.listdir(subdir_path):
                    img_path = os.path.join(subdir_path, img_name)
                    if img_name.lower().endswith((".png", ".jpg", ".jpeg")):
                        self.image_paths.append(img_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)
   
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.float32)

# Define Preprocessing
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Test Data
test_dataset = DeepfakeDataset(root_dir="PATH TO TEST DATASET/Test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Evaluate Model
y_true, y_pred, y_scores = [], [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).squeeze()
        preds = (outputs > 0.5).float()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_scores.extend(outputs.cpu().numpy())

# Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Real", "Fake"]))

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=["Real", "Fake"], yticklabels=["Real", "Fake"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# ROC Curve
fpr, tpr, _ = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


## Save the Metrics

In [ ]:
from sklearn.metrics import classification_report

# Save classification report
report = classification_report(y_true, y_pred, target_names=["Real", "Fake"])
with open("classification_report.txt", "w") as f:
    f.write(report)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot and save confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=["Real", "Fake"], yticklabels=["Real", "Fake"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix.png", dpi=300)  # Save image
plt.close()

In [ ]:
from sklearn.metrics import roc_curve, auc

# Compute ROC curve and AUC
fpr, tpr, _ = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)

# Plot and save ROC curve
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.savefig("roc_curve.png", dpi=300)  # Save image
plt.close()


# Total Sample Dataset

In [ ]:
import os

# Function to count images in subdirectories
def count_images(root_folder):
    counts = {}

    for split in ["Train", "Test", "Val"]:  # Main subfolders
        split_path = os.path.join(root_folder, split)

        if not os.path.exists(split_path):
            print(f"Skipping {split_path}, folder not found.")
            continue

        for category in ["Real", "Fake"]:  # Child subfolders
            category_path = os.path.join(split_path, category)

            if os.path.exists(category_path):
                image_count = len([f for f in os.listdir(category_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))])
                counts[f"{split}/{category}"] = image_count
            else:
                print(f"Skipping {category_path}, folder not found.")

    return counts

# Define root folder path
root_folder = "new_sample_1.8L"  # Change this to your actual path

# Get image counts
image_counts = count_images(root_folder)

# Print results
for folder, count in image_counts.items():
    print(f"{folder}: {count} images")
